# Dataset Cleaning 

The main objectives of this notebook are: 
- Import dataset 
- Get familiar with the data 
- Determine if there are data quality issues 
- Resolve any data quality issues 

In [169]:
# %load_ext autoreload
# %autoreload 2

In [170]:
import polars as pl 
from google.cloud import bigquery
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import sys 
import os 


# Manually add path to read from another folder 
path2add = os.path.normpath(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir, "utils"))
)
if not (path2add in sys.path):
    sys.path.append(path2add)

from preprocessing import *

In [171]:
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [172]:
# Create client object
client = bigquery.Client()

In [173]:
QUERY = ("SELECT `date`, `visitStartTime`, `fullVisitorId`, `geoNetwork`.`subContinent`, `geoNetwork`.`country`,"
        "`geoNetwork`.`city`,`device`.`browser`, `device`.`operatingSystem`,`device`.`deviceCategory`,"
        "`trafficSource`.`source`, `h`.`item`.`transactionId`,`hp`.`v2ProductName`,`hp`.`v2ProductCategory` "
        "FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170731`, UNNEST(hits) AS h,"
        "UNNEST(h.product) AS hp;")

In [174]:
run_query = client.query(QUERY)

In [175]:
df = pl.from_arrow(run_query.to_arrow())

In [176]:
df.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [177]:
df.shape

(49512, 13)

In [178]:
df.columns

['date',
 'visitStartTime',
 'fullVisitorId',
 'subContinent',
 'country',
 'city',
 'browser',
 'operatingSystem',
 'deviceCategory',
 'source',
 'transactionId',
 'v2ProductName',
 'v2ProductCategory']

In [179]:
df.sample()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501527887,"""0120744311326673223""","""Northern Europe""","""United Kingdom""","""not available in demo dataset""","""Chrome""","""Macintosh""","""desktop""","""google""",null,"""YouTube Men's Short Sleeve Her…","""Home/Shop by Brand/YouTube/"""


In [180]:
df.describe()

statistic,date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,str,f64,str,str,str,str,str,str,str,str,str,str,str
"""count""","""49512""",49512.0,"""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""462""","""49512""","""49512"""
"""null_count""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""49050""","""0""","""0"""
"""mean""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,21502.167394,null,null,null,null,null,null,null,null,null,null,null
"""min""","""20170731""",1.5015e9,"""0002457163364254438""","""Australasia""","""Algeria""","""(not set)""","""Amazon Silk""","""(not set)""","""desktop""","""(direct)""","""ORD201707311786""","""1 oz Hand Sanitizer""","""${escCatTitle}"""
"""25%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""max""","""20170731""",1.5016e9,"""9997362993085245352""","""Western Europe""","""Vietnam""","""not available in demo dataset""","""YaBrowser""","""iOS""","""tablet""","""youtube.com""","""ORD201707312663""","""YouTube Youth Short Sleeve Tee…","""Wearables/Men's T-Shirts/"""


In [181]:
df.select(pl.all().is_null().sum())

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,49050,0,0


In [182]:
df.filter(pl.col('transactionId') != 'null')

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Blackout Cap""","""Headgear"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Women's Vintage Hero Te…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Men's Long & Lean Tee C…","""Apparel"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Google 25 oz Red Stainless Ste…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android 17oz Stainless Steel S…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android Men's Short Sleeve Her…","""Apparel"""


In [183]:
df_cleaned = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
df_cleaned.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,str,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [184]:
df_cleaned = df_cleaned.with_columns(pl.when(pl.col('transactionId') != 'null').then(1).otherwise(0).alias("transactionId"))

# Encoding the df

In [185]:
encoded_df = target_encoding(df)
encoded_df

fullVisitorId,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductCategory
str,f64,f64,f64,f64,f64,f64,i32,f64
"""2219384770970157334""",0.0,0.006208,0.001795,0.005962,0.011631,0.003665,0,0.0
"""2219384770970157334""",0.0,0.006208,0.001795,0.005962,0.011631,0.003665,0,0.0
"""2219384770970157334""",0.0,0.006208,0.001795,0.005962,0.011631,0.003665,0,0.0
"""2219384770970157334""",0.0,0.006208,0.001795,0.005962,0.011631,0.003665,0,0.0
"""2219384770970157334""",0.0,0.006208,0.001795,0.005962,0.011631,0.003665,0,0.0
…,…,…,…,…,…,…,…,…
"""7483600664917507409""",0.014312,0.013889,0.011785,0.01692,0.011631,0.020215,1,0.254777
"""7483600664917507409""",0.014312,0.013889,0.011785,0.01692,0.011631,0.020215,1,0.262
"""7483600664917507409""",0.014312,0.013889,0.011785,0.01692,0.011631,0.020215,1,0.262


In [186]:
# encoded_df = label_encoding(df)
# encoded_df

In [187]:
user_profiles = encoded_df.group_by(pl.col('fullVisitorId')).agg(
    pl.col('country'), pl.col('city'), 
    pl.col('browser'),pl.col('operatingSystem'),pl.col('deviceCategory'),
    pl.col('source'),pl.col('transactionId'),pl.col('v2ProductCategory')
)
user_profiles

fullVisitorId,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductCategory
str,list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[i32],list[f64]
"""6253636989597827077""","[0.014312, 0.014312, … 0.014312]","[0.006208, 0.006208, … 0.006208]","[0.011785, 0.011785, … 0.011785]","[0.008481, 0.008481, … 0.008481]","[0.011631, 0.011631, … 0.011631]","[0.003665, 0.003665, … 0.003665]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""6036322103240164915""","[0.014312, 0.014312]","[0.043614, 0.043614]","[0.011785, 0.011785]","[0.018405, 0.018405]","[0.011631, 0.011631]","[0.020215, 0.020215]","[0, 0]","[0.0, 0.0]"
"""3221924345288041502""","[0.014312, 0.014312, … 0.014312]","[0.013889, 0.013889, … 0.013889]","[0.011785, 0.011785, … 0.011785]","[0.018405, 0.018405, … 0.018405]","[0.011631, 0.011631, … 0.011631]","[0.0, 0.0, … 0.0]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""0280589015308186459""","[0.014312, 0.014312, … 0.014312]","[0.078788, 0.078788, … 0.078788]","[0.011785, 0.011785, … 0.011785]","[0.01692, 0.01692, … 0.01692]","[0.011631, 0.011631, … 0.011631]","[0.020215, 0.020215, … 0.020215]","[0, 0, … 1]","[0.0, 0.0, … 0.262]"
"""3496145444042098592""","[0.014312, 0.014312, … 0.014312]","[0.006208, 0.006208, … 0.006208]","[0.000797, 0.000797, … 0.000797]","[0.000566, 0.000566, … 0.000566]","[0.003792, 0.003792, … 0.003792]","[0.003665, 0.003665, … 0.003665]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
…,…,…,…,…,…,…,…,…
"""0530654477551060890""","[0.014312, 0.014312, … 0.014312]","[0.014785, 0.014785, … 0.014785]","[0.000797, 0.000797, … 0.000797]","[0.000566, 0.000566, … 0.000566]","[0.003792, 0.003792, … 0.003792]","[0.003665, 0.003665, … 0.003665]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""4044743325936001564""","[0.014312, 0.014312, … 0.014312]","[0.003949, 0.003949, … 0.003949]","[0.011785, 0.011785, … 0.011785]","[0.018405, 0.018405, … 0.018405]","[0.011631, 0.011631, … 0.011631]","[0.020215, 0.020215, … 0.020215]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""0945021702933031829""","[0.0, 0.0, … 0.0]","[0.006208, 0.006208, … 0.006208]","[0.011785, 0.011785, … 0.011785]","[0.006482, 0.006482, … 0.006482]","[0.003792, 0.003792, … 0.003792]","[0.003665, 0.003665, … 0.003665]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"


In [188]:
user_profiles = user_profiles.filter(pl.col('fullVisitorId') != '327480040583975209')

In [189]:
user_profiles_no_id = user_profiles.drop('fullVisitorId')
user_profiles_no_id = user_profiles_no_id.to_numpy()
user_profiles_no_id

array([[array([0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195

In [190]:
active_users = encoded_df.drop('fullVisitorId')
active_users = active_users.to_numpy()
active_users

array([[0.        , 0.00620763, 0.00179533, ..., 0.00366492, 0.        ,
        0.        ],
       [0.        , 0.00620763, 0.00179533, ..., 0.00366492, 0.        ,
        0.        ],
       [0.        , 0.00620763, 0.00179533, ..., 0.00366492, 0.        ,
        0.        ],
       ...,
       [0.01431195, 0.01388889, 0.01178473, ..., 0.02021548, 1.        ,
        0.262     ],
       [0.01431195, 0.01388889, 0.01178473, ..., 0.02021548, 1.        ,
        0.262     ],
       [0.01431195, 0.01388889, 0.01178473, ..., 0.02021548, 0.        ,
        0.        ]])

In [191]:
test_users = list(encoded_df.filter(pl.col('fullVisitorId') == '327480040583975209').row(-1))[1:]
test_users

[0.0,
 0.0,
 0.0017953321364452424,
 0.0059616381544841885,
 0.011630818887559475,
 0.0,
 0,
 0.0]

In [192]:
similarities = calculate_similarity(user_profiles, user_profiles_no_id, test_users)
similarities

{'6253636989597827077': 0.6390198441146631,
 '6036322103240164915': 0.3606959205912358,
 '3221924345288041502': 0.6346243619758428,
 '0280589015308186459': 0.16410707060440785,
 '3496145444042098592': 0.22474107068524377,
 '6511496565431647115': 0.8775347920156277,
 '3697249728510765050': 0.6025546497713562,
 '3951537161592094783': 0.3704339537717871,
 '8230445621024438058': 0.22474107068524382,
 '8738860778608972574': 0.6822559276164731,
 '8330772482336842742': 0.6333125560967926,
 '033597019173782414': 0.49075278414433837,
 '2800236834999512233': 0.7798026663496941,
 '5581888378481133736': 0.22474107068524385,
 '8805708485521378179': 0.4519185173345008,
 '3363762096993702886': 0.5601262055828554,
 '3094733780828697908': 0.8270208718328739,
 '3565982486935193176': 0.6247637108895333,
 '0354546464926350593': 0.5355208131958746,
 '3830485242574967009': 0.6257154887759051,
 '3456285538294788190': 0.09971452787088139,
 '1478443307193100229': 0.7651852963288334,
 '6177044399356122277': 0.3

In [193]:
sorted_dict = sorted(similarities.items(), key=lambda x: x[1])
sorted_dict[-5:]

[('4682317243688313274', 0.9635111916051311),
 ('4526419438777992681', 0.9635111916051312),
 ('9306329208927979354', 0.9897115733342824),
 ('8028335945205964246', 0.9897115733342824),
 ('8302271511945065814', 0.990696750000991)]

In [ ]:
actual_bought = df.filter((pl.col('fullVisitorId') == '0345672699449577691') & (pl.col('transactionId') != 'null')).select(pl.col('v2ProductCategory')).to_series().to_list()
actual_bought 

['Headgear',
 'Apparel',
 'Apparel',
 'Apparel',
 'Headgear',
 'Apparel',
 'Apparel',
 'Apparel']

In [195]:
rec = recommendation(df,sorted_dict)
rec

['Apparel', 'Apparel', 'Electronics', 'Apparel', 'Apparel']

In [196]:
precision(actual_bought, rec, 3)

0.5

In [197]:
df.write_csv("../data/cleaned_google_analytics.csv")

### Data Pre-processing Pipeline

In [198]:
def preprocessing_pipeline(df: pl.DataFrame) -> pl.DataFrame:
    """
    Args:
        df : pl.DataFrame

    Returns:
        pl.DataFrames: pre-processed dataframe
    """

    df = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
    encoded_df = encoding(df)
    
    user_profiles = encoded_df.group_by(pl.col('fullVisitorId')).agg(
        pl.col('country'), pl.col('browser'),
        pl.col('transactionId'), pl.col('v2ProductName')
    )
    user_profiles_no_id = user_profiles.drop('fullVisitorId')
    user_profiles_no_id = user_profiles_no_id.to_numpy()
    active_users = encoded_df.drop('fullVisitorId')
    active_users = active_users.to_numpy()

    similarities = calculate_similarity(user_profiles, user_profiles_no_id, active_users)
    sorted_dict = sorted(similarities.items(), key=lambda x: x[1])

    
    df.write_csv("../data/cleaned_google_analytics.csv")
    return recommendation(df,sorted_dict) 

In [199]:
# preprocessing_pipeline(df)

# Conclusions 
- 